In [6]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import time
import sys
import os
import csv
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
service = Service(
    r"/path/to/geckodriver"
)
driver = webdriver.Firefox(service=service, options=options)

In [26]:
actions = ActionChains(driver)

In [27]:
driver.get("https://www.instagram.com/")
sleep(3)

In [28]:
# enter username
username = driver.find_element(By.NAME, "username")
username.send_keys(os.getenv("INSTA_PASSWORD"))
sleep(2)

# enter password
password = driver.find_element(By.NAME, "password")
password.send_keys(os.getenv("INSTA_PASSWORD"))
sleep(1)
password.submit()
sleep(5)

In [36]:
reels_url = "https://www.instagram.com/reels/"
driver.get(reels_url)
sleep(3)

In [37]:
# CSV file setup
csv_filename = "instagram_reels_links.csv"
csv_header = ["insta_links"]
if not os.path.exists(csv_filename):
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(csv_header)

In [38]:
def focus_on_reel():
    sleep(2)
    try:
        window_size = driver.get_window_size()
        center_x = window_size['width'] // 2
        center_y = window_size['height'] // 2

        actions.move_by_offset(center_x, center_y).click().perform()
        sleep(1)
        print("Focused on the first reel.")
    except Exception as e:
        print("Could not focus on reel:", e)

focus_on_reel()

Focused on the first reel.


In [39]:
def scrape_reels_links(scrolls=50, delay=0.5):
    collected_links = set()
    for _ in tqdm(range(scrolls), desc="Scrolling through Reels"):
        current_url = driver.current_url
        if "instagram.com/reels/" in current_url and current_url not in collected_links:
            collected_links.add(current_url)
            with open(csv_filename, "a", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow([current_url])
    
        # Scroll to the next reel
        actions.send_keys(Keys.ARROW_DOWN).perform()
        sleep(delay)

    print(f"Scraped {len(collected_links)} unique reels links.")

In [40]:
%%time
scrape_reels_links(1500, 0.5)

Scrolling through Reels: 100%|██████████████| 1500/1500 [17:35<00:00,  1.42it/s]

Scraped 943 unique reels links.
CPU times: user 4.74 s, sys: 553 ms, total: 5.29 s
Wall time: 17min 35s


In [5]:
# # Close browser
# driver.quit()